In [5]:
import csv
import sys
import types
import shutil
import pydicom
import os
import os.path
import subprocess
import numpy as np
import pandas as pd
import nibabel as nib
import pyautogui
from glob import glob
from PIL import Image

import zipfile
from zipfile import ZipFile

STEP 1: this first code is intended to anonimize your DICOM files. In case you want to remove information like the names, birth date, among others.

In [ ]:
def anonymize_dicom_tags(dataset):
    # List of DICOM tags to be removed for anonymization
    delete_phi_tags = [
        (0x0010, 0x0010),  # Patient's Name
        (0x0010, 0x0020),  # Patient ID
        (0x0010, 0x0030),  # Patient's Birth Date
        (0x0010, 0x1010),  # Patient's Age
        (0x0010, 0x1020)   # Patient's Size
    ]

    # Remove identified tags
    for tag in delete_phi_tags:
        if tag in dataset:
            del dataset[tag]

    # Remove file meta information
    if hasattr(dataset, "file_meta"):
        dataset.file_meta.clear()
    return dataset

def getanonfiles(anonymizePath):
    for directorypath, directorynames, filenames in os.walk(anonymizePath):
        for filename in filenames:
            if filename.endswith('.IMA') or filename.endswith('.dcm'):
                selectFile = os.path.join(directorypath, filename)
                dataset = pydicom.read_file(selectFile, force=True)
                dataset = anonymize_dicom_tags(dataset)
                dataset.save_as(selectFile)

    print('--> Task completed.')

anonymizePath = 'path/to/raw/DICOM'
getanonfiles(anonymizePath)

STEP 2: Conversion from DICOM to NIfTI (need to install dcm2niix w/ pip)

In [ ]:
#required folders name : "path/to/dicom" + "participant Anonymisation Code"

data = pyautogui.prompt('Which data do want to convert?')

input_directory = "path/to/raw/DICOM"

def convert_dicom_series(input_directory):
    #for files in os.walk(input_directory):
    #Herebefore Only if all subjects at the same time
    # You might need to install 'dcm2niix'
    command = ["dcm2niix", 'sub-','-f', data+'_%p_%d', '-s', 'y', '-v', '2', input_directory]

    # old ; command = ["dcm2niix", 'sub-','-f', data+'_%p_%s', '-s', 'y', root]

    # Execute the command
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()

    # If you want to delete the original DICOM files, uncomment these lines
    #for file in files:
        #os.remove(os.path.join(root, file))

# Call the function to convert DICOM series
convert_dicom_series(input_directory)
print(data)

Step 3 - If names are not correct : rename folders with correct BIDS names

In [ ]:
rootdir = r"Bids/Directory"

# Define a prefix to be added to the new names
new_prefix = 'sub-'

counter = 1
renaming_dict = {}

for subdir in os.listdir(rootdir):
    subdir_path = os.path.join(rootdir, subdir)
    new_name = f'{new_prefix}{counter:02d}'
    
    os.rename(subdir_path, os.path.join(rootdir, new_name))

    renaming_dict[subdir] = new_name
    counter += 1

# Print the alterations
for old_name, new_name in renaming_dict.items():
    print(f'{old_name} -> {new_name}')

STEP 4 :  Rename files to be compatible with BIDS format

In [ ]:
#The transformed BIDS data must be in a directory named as the participant's anonymisation code (e.g. sub-001)

data = pyautogui.prompt('Which data do want to convert?')

input_directory = "Directory/BIDS/Files"+ data

filenames = os.listdir(input_directory)

# Iterate through each filename
for count, filename in enumerate(filenames):
    elements = filename
    #func - task
        #olhos fechados
    if "ep2d_bold_moco_olhosfechados_resting" in elements and "MoCo_bas_MoCoSeries" in elements:
        new_filename = data+"_task-rest_acq-MoCo_bold"  # Replace with your desired new name
    
    elif "ep2d_bold_moco_olhosfechados_resting" in elements and "Design" in elements:
        new_filename = data+"_task-rest_acq-Design_bold"  # Replace with your desired new name

    elif "_ep2d_bold_moco_olhosfechados_resting_ep2d_bold_moco_olhosfechados_resting" in elements :
        new_filename = data+"_task-rest_acq-Raw_bold"  # Replace with your desired new name

    elif "_ep2d_bold_moco_olhosfechados_resting" in elements and "EvaSeries_GLM" in elements:
        new_filename = data+"_task-rest_acq-EvaGLM_bold"  # Replace with your desired new name

    elif "_ep2d_bold_moco_olhosfechados_resting" in elements and "intermediate_t-Map" in elements:
        new_filename = data+"_task-rest_acq-InterT_bold"  # Replace with your desired new name

    elif "_ep2d_bold_moco_olhosfechados_resting" in elements and "Mean_&_t-Maps" in elements:
        new_filename = data+"_task-rest_acq-MeanT_bold"  # Replace with your desired new name

    elif "_ep2d_bold_moco_olhosfechados_resting" in elements and "StartFMRI" in elements:
        new_filename = data+"_task-rest_acq-Start_bold"  # Replace with your desired new name


        #tarefa
    elif "ep2d_bold_moco_tarefa" in elements and "MoCo_bas_MoCoSeries" in elements:
        new_filename = data+"_task-tarefa_acq-MoCo_bold"  # Replace with your desired new name

    elif "ep2d_bold_moco_tarefa" in elements and "MoCo_bas_MoCoSeries" in elements:
        new_filename = data+"_task-tarefa_acq-MoCo_bold"  # Replace with your desired new name
    
    elif "ep2d_bold_moco_tarefa" in elements and "Design" in elements:
        new_filename = data+"_task-tarefa_acq-Design_bold"  # Replace with your desired new name
    
    elif "ep2d_bold_moco_tarefa_ep2d_bold_moco_tarefa" in elements :
        new_filename = data+"_task-tarefa_acq-Raw_bold"  # Replace with your desired new name
    
    elif "ep2d_bold_moco_tarefa" in elements and "EvaSeries_GLM" in elements:
        new_filename = data+"_task-tarefa_acq-EvaGLM_bold"  # Replace with your desired new name
    
    elif "ep2d_bold_moco_tarefa" in elements and "intermediate_t-Map" in elements:
        new_filename = data+"_task-tarefa_acq-InterT_bold"  # Replace with your desired new name
    
    elif "ep2d_bold_moco_tarefa" in elements and "Mean_&_t-Maps" in elements:
        new_filename = data+"_task-tarefa_acq-MeanT_bold"  # Replace with your desired new name
    
    elif "ep2d_bold_moco_tarefa" in elements and "StartFMRI" in elements:
        new_filename = data+"_task-tarefa_acq-Start_bold"  # Replace with your desired new name

    #anat
    elif "t1_mpr_sag_p2_iso_" in elements :
        new_filename = data+"_T1w"  # Replace with your desired new name

    #fmap
    elif "gre_field_mapping_" in elements :
        new_filename = data+"_fieldmap"  # Replace with your desired new name

    #dwi
    elif "ep2d_diff_MDDW_64_p2_ep2d" in elements :
        new_filename = data+"_dwi"  # Replace with your desired new name
    
    elif "ep2d_diff_MDDW_64_b0_PA_p2" in elements :
        new_filename = data+"_acq-PA_dwi"  # Replace with your desired new name
    
    elif "ep2d_diff_MDDW_64_b0_AP_p2_" in elements :
        new_filename = data+"_acq-AP_dwi"  # Replace with your desired new name

    else:
        new_filename = filename  # No matching combination, keep the original name

    # Rename the file
    print(type(elements))
    print(elements)
    print(new_filename)
    print(filename)
    _, extension = os.path.splitext(filename)
    new_filename_extension = new_filename + extension
    os.rename(os.path.join(input_directory, filename), os.path.join(input_directory, new_filename_extension))




Get header info of a specific file to check if the data are complete

In [ ]:
import pydicom

data = "path/to/specific/nii/file"
dcm_data = pydicom.read_file(data)
print(dcm_data)
from nibabel.nicom import csareader as csar
csa = csar.get_csa_header(dcm_data)
print(csa)
#ascii_header = csa['tags']['ep2d_bold_moco_tarefa']['items'][0]

Check sfrom and qform matrices to check if the data were correctly converted 

In [ ]:
import nibabel as nib
# Load NIfTI file
nifti_img = nib.load("path/to/specific/nii/file")
# Access header information
header = nifti_img.header

print(header)
print(nifti_img.get_qform(coded=True))
print(nifti_img.get_sform(coded=True))

Modify a specific dimension if data are incomplete or incorrect due to an error during conversion (e.g. problem with 4D matrix conversion here, leading to an incorrect number of dimensions )

In [ ]:
import nibabel as nib

# Load the NIfTI file
nifti_file_path = ("path/to/specific/nii/file")
img = nib.load(nifti_file_path)

# Access the header
header = img.header

# Modify the dim[3] value
header['dim'][3] = 50


# Save the modified NIfTI file
nib.save(img, 'name of the nii file')

Specific to the 4D conversion problem due to incomplete header info :
Add dicom tags corresponding to slice number ((0019,100A), Number Of Images In Mosaic) to enable a correct conversion for functional data



In [3]:
import pydicom
from pydicom import dcmread
import os
dicom_paths = "path/to/the/incomplete/dicom" #copy of the original raw dicom!

for s in os.listdir(dicom_paths):
    ds = dcmread((dicom_paths + "/" + s), force  = True)
    ds.add_new([0x0019,0x100A], 'US', 50) # cat. "US"
    ds.save_as(dicom_paths + "/" + s)
    

read dicom to check info 

In [ ]:
import pydicom
from pydicom import dcmread
import os
dicom_paths ="path/to/the/now/completed/dicom"
for s in os.listdir(dicom_paths):
    ds = dcmread((dicom_paths + "/" + s), force  = True)
    print(ds)

Another way to read CSA header from DICOM 

In [ ]:
from dicom_parser import Image
from dicom_parser.utils.siemens.csa.header import CsaHeader

image = Image("path/to/any/dicom")
csa = image.header.get("CSAImageHeaderInfo")
raw_csa = image.header.get("CSAImageHeaderInfo", parsed=False)

print(csa)

